In [357]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import numpy as np
from gap_statistic import OptimalK
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score
import os
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"
os.environ["OMP_NUM_THREADS"] = '2'
df = pd.read_excel("data4.xlsx", header=0)
df["Date"] = pd.to_datetime(df["Date"])
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Unnamed: 2,Date,Contest number,Word,Number of reported results,Number in hard mode,1 try,...,6 tries,7 or more tries (X),ratio,vowel,len,uniq,star,three_s,three_ss,three_sss
0,0,0,NaN,NaN,2022-01-07,202,slump,-0.117038,1362,0.684682,...,-0.416685,-0.438101,0.016892,-1.278686,5,0.604574,-0.517873,0.27,27,0.030310
1,1,1,NaN,NaN,2022-01-08,203,crank,0.115919,1763,0.684682,...,0.390756,-0.196601,0.017369,-1.278686,5,0.604574,-0.517873,0.29,29,0.531068
2,2,2,NaN,NaN,2022-01-09,204,gorge,0.004022,1913,0.684682,...,1.682660,0.286398,0.020912,0.348316,5,-1.420179,-1.410324,0.17,17,-1.258570
3,3,3,NaN,NaN,2022-01-10,205,query,0.178765,2242,0.684682,...,0.875220,-0.196601,0.020927,0.348316,5,0.604574,-0.517873,0.21,21,-0.621527
4,4,4,NaN,NaN,2022-01-11,206,drink,0.700481,3017,0.684682,...,-1.062637,-0.438101,0.019606,-1.278686,5,0.604574,1.267029,0.45,45,3.079240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,350,350,NaN,NaN,2022-12-27,556,condo,-0.783899,2012,-0.594592,...,0.390756,0.044899,0.096365,0.348316,5,-1.420179,-1.410324,0.19,19,-2.272671
351,351,351,NaN,NaN,2022-12-28,557,impel,-0.791923,1937,-0.594592,...,-0.416685,-0.438101,0.096081,0.348316,5,0.604574,-0.517873,0.24,24,-1.506740
352,352,352,NaN,NaN,2022-12-29,558,havoc,-0.793698,1919,-0.594592,...,0.067779,-0.196601,0.095945,0.348316,5,0.604574,0.374578,0.18,18,-2.401559
353,353,353,NaN,NaN,2022-12-30,559,molar,-0.780272,1973,-0.594592,...,-0.416685,-0.438101,0.093048,0.348316,5,0.604574,-0.517873,0.25,25,-1.256361


In [358]:
df.iloc[:, 9:16]

,1 try,2 tries,3 tries,4 tries,5 tries,6 tries,7 or more tries (X)
0,0.684682,-0.696124,0.041752,1.124468,0.053344,-0.416685,-0.438101
1,0.684682,-0.195366,0.041752,-0.367817,0.053344,0.390756,-0.196601
2,0.684682,-0.696124,-1.247128,-1.113959,1.067829,1.682660,0.286398
3,0.684682,-0.445745,-0.860464,-0.554352,1.067829,0.875220,-0.196601
4,0.684682,0.806150,1.588408,0.191790,-1.299303,-1.062637,-0.438101
...,...,...,...,...,...,...,...
350,-0.594592,-0.946503,-0.731576,0.378326,0.898748,0.390756,0.044899
351,-0.594592,-0.696124,-0.216024,1.311004,0.222425,-0.416685,-0.438101
352,-0.594592,-0.946503,-0.860464,0.937933,1.067829,0.067779,-0.196601
353,-0.594592,-0.445745,-0.216024,0.937933,0.391506,-0.416685,-0.438101


In [359]:
mu = df["three_ss"].mean()
st = df["three_ss"].std()
df["three_ss"] = (df["three_ss"] - mu) / st
# [10, -3.4449, -0.5179]
eerie = pd.DataFrame({
    "three_ss": [(10 - mu) / st],
    "uniq": [-3.4449],
    "star": [-0.5179]
})
eerie

,three_ss,uniq,star
0,-1.651122,-3.4449,-0.5179


In [360]:
pd.concat((df["three_ss"], df["uniq"], df["star"]), axis=1)

,three_ss,uniq,star
0,-0.167644,0.604574,-0.517873
1,0.006883,0.604574,-0.517873
2,-1.040278,-1.420179,-1.410324
3,-0.691225,0.604574,-0.517873
4,1.403097,0.604574,1.267029
...,...,...,...
350,-0.865751,-1.420179,-1.410324
351,-0.429434,0.604574,-0.517873
352,-0.953015,0.604574,0.374578
353,-0.342171,0.604574,-0.517873


In [361]:
# data = pd.concat((df.iloc[:, 9:16], df["uniq"], df["star"]), axis=1)
data = pd.concat((df["three_ss"], df["uniq"], df["star"]), axis=1)
data = data.append(eerie, ignore_index=True)
data

,three_ss,uniq,star
0,-0.167644,0.604574,-0.517873
1,0.006883,0.604574,-0.517873
2,-1.040278,-1.420179,-1.410324
3,-0.691225,0.604574,-0.517873
4,1.403097,0.604574,1.267029
...,...,...,...
351,-0.429434,0.604574,-0.517873
352,-0.953015,0.604574,0.374578
353,-0.342171,0.604574,-0.517873
354,-0.865751,0.604574,-0.517873


In [362]:
data = data.astype(float)
data = np.array(data)
data

array([[-0.16764407,  0.60457439, -0.51787291],
       [ 0.00688275,  0.60457439, -0.51787291],
       [-1.04027816, -1.42017945, -1.4103238 ],
       ...,
       [-0.34217089,  0.60457439, -0.51787291],
       [-0.86575134,  0.60457439, -0.51787291],
       [-1.65112202, -3.4449    , -0.5179    ]])

In [363]:
data[-1]

array([-1.65112202, -3.4449    , -0.5179    ])

In [364]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import LocallyLinearEmbedding

pca = PCA(n_components=2)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)

data = pca.fit_transform(data)
# data = lle.fit_transform(data)

data

array([[-5.18354616e-02, -7.60630222e-01],
       [-1.70575636e-01, -7.65358656e-01],
       [ 2.13841687e+00, -4.04183925e-01],
       [ 3.04385061e-01, -7.46444918e-01],
       [-1.84975191e+00,  7.02633986e-01],
       [ 1.83315951e+00,  3.51090352e-01],
       [ 3.72162065e-01, -1.51117606e+00],
       [-1.48512447e+00, -5.02760732e-02],
       [-9.42386767e-01, -7.96093481e-01],
       [-8.83016680e-01, -7.93729264e-01],
       [ 3.55348230e-01,  2.30146626e-02],
       [-2.74871013e+00,  1.43426609e+00],
       [ 1.62390160e-01,  1.05198763e+00],
       [-7.64276506e-01, -7.89000829e-01],
       [ 3.04385061e-01, -7.46444918e-01],
       [-1.02798631e-01, -1.53008980e+00],
       [ 2.01126978e+00,  3.58183004e-01],
       [-7.13313337e-01, -1.95412485e-02],
       [-5.18354616e-02, -7.60630222e-01],
       [-1.12890395e+00, -3.60907695e-02],
       [ 6.09642414e-01, -1.50171919e+00],
       [-1.73941865e+00,  1.47445778e+00],
       [ 3.63755148e-01, -7.44080701e-01],
       [-1.

In [365]:
K = np.arange(2, 11)
dot_color = "#7bcc79"

In [366]:
# # 使用 OptimalK 类进行 Gap Statistic 计算
# optimalK = OptimalK(n_jobs=4, parallel_backend='multiprocessing')
#
# n_clusters = optimalK(data, cluster_array=np.arange(2, 10))
#
# n_clusters

In [367]:
# gap = optimalK.gap_df
# gap

In [368]:
# index = np.argmax(gap["gap_value"])
# fig = px.line(gap, x="n_clusters", y="gap_value")
# fig.add_scatter(
#     x=[gap["n_clusters"][index]],
#     y=[gap["gap_value"][index]],
#     mode="markers",
#     marker=dict(
#         size=10,
#         color=dot_color
#     ),
# )
# fig.update_layout(
#     showlegend=False,
#     width=800,
#     height=500,
#     dict1=dict(
#         title=dict(
#             text="The Result of Gap Statistic",
#             font=dict(
#                 size=20
#             ),
#             x=0.48,  # 0 ~ 1
#             xanchor="center",
#             xref="container",
#             y=0.95,
#             yanchor="middle"
#         ),
#     ),
# )
# fig.update_xaxes(
#     tickvals=K,
# )

In [369]:
# fig.write_image("gap4.svg")

In [370]:
# meanDispersions = []
# for k in K:
#     kemans = KMeans(n_clusters=k)
#     kemans.fit(data)
#     #计算平均离差
#     m_Disp = sum(np.min(cdist(data, kemans.cluster_centers_, 'euclidean'), axis=1)) / data.shape[0]
#     meanDispersions.append(m_Disp)
#
# plt.rcParams['font.family'] = ['sans-serif']
# plt.rcParams['font.sans-serif'] = ['SimHei']  #使折线图显示中文
# plt.plot(K, meanDispersions, 'bx-')
# plt.xlabel('k')
# plt.ylabel('平均离差')
# plt.title('用肘部方法选择K值')
# plt.show()

In [371]:
# meanDispersions[0], meanDispersions[2], meanDispersions[3], meanDispersions[4]

In [372]:
# meanDispersions[0] = 0.047275882922132565

In [373]:
# fig = px.line(x=K, y=meanDispersions)
# fig.add_scatter(
#     x=K, y=meanDispersions,
#     mode="markers",
#     marker=dict(
#         size=10
#     ),
# ),
#
# fig.add_scatter(
#     x=[6], y=[meanDispersions[4]],
#     mode="markers",
#     marker=dict(
#         size=10,
#         color=dot_color
#     ),
# ),
# fig.update_layout(
#     showlegend=False,
#     width=800,
#     height=500,
#     dict1=dict(
#         title=dict(
#             text="The Result of Elbow Method",
#             font=dict(
#                 size=20
#             ),
#             x=0.48,  # 0 ~ 1
#             xanchor="center",
#             xref="container",
#             y=0.95,
#             yanchor="middle"
#         ),
#     ),
# )
# fig.update_xaxes(
#     tickvals=K,
#     title="n_clusters"
# )
# fig.update_yaxes(
#     title="Distortion"
# )
# fig

In [374]:
# fig.write_image("dis4.svg")

In [375]:
# for i in range(2, 10):
#     kmeans = KMeans(n_clusters=i, max_iter=500)
#     train_clusters = kmeans.fit_predict(data)
#     si_score = metrics.silhouette_score(data, kmeans.labels_)
#     ch_score = metrics.calinski_harabasz_score(data, kmeans.labels_)
#     db_score = davies_bouldin_score(data, kmeans.labels_)
#     print(i)
#     print(si_score)
#     print(ch_score)
#     print(db_score)
#     print("---------")

In [376]:
kmeans = KMeans(n_clusters=6, max_iter=500)
train_clusters = kmeans.fit_predict(data)

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.



In [377]:
pd.value_counts(train_clusters)

0    109
5     91
2     58
3     55
4     36
1      7
dtype: int64

In [378]:
# eerie = np.array([, [10, -3.4449, -0.5179]])
# eerie = pca.fit_transform(eerie)
# eerie

In [379]:
data.shape, train_clusters.shape

((356, 2), (356,))

In [380]:
data = np.c_[data, train_clusters]
data = pd.DataFrame(data)
data

,0,1,2
0,-0.051835,-0.760630,5.0
1,-0.170576,-0.765359,5.0
2,2.138417,-0.404184,2.0
3,0.304385,-0.746445,5.0
4,-1.849752,0.702634,3.0
...,...,...,...
351,0.126275,-0.753538,5.0
352,0.117868,0.013558,5.0
353,0.066905,-0.755902,5.0
354,0.423125,-0.741716,5.0


In [381]:
data

,0,1,2
0,-0.051835,-0.760630,5.0
1,-0.170576,-0.765359,5.0
2,2.138417,-0.404184,2.0
3,0.304385,-0.746445,5.0
4,-1.849752,0.702634,3.0
...,...,...,...
351,0.126275,-0.753538,5.0
352,0.117868,0.013558,5.0
353,0.066905,-0.755902,5.0
354,0.423125,-0.741716,5.0


In [382]:
fig = go.Figure()


fig.add_scatter(x=data[data[2] == 4][0], y=data[data[2] == 4][1], mode="markers", name="A")

fig.add_scatter(x=data[data[2] == 0][0], y=data[data[2] == 0][1], mode="markers", name="B")
fig.add_scatter(x=data[data[2] == 5][0], y=data[data[2] == 5][1], mode="markers", name="C")
fig.add_scatter(x=data[data[2] == 2][0], y=data[data[2] == 2][1], mode="markers", name="D")
fig.add_scatter(x=data[data[2] == 1][0], y=data[data[2] == 1][1], mode="markers", name="E")

fig.add_scatter(x=data[data[2] == 3][0], y=data[data[2] == 3][1], mode="markers", name="F")
fig.update_layout(
    dict1=dict(
        title=dict(
            text="The Result of Classification",
            font=dict(
                size=20
            ),
            x=0.48,  # 0 ~ 1
            xanchor="center",
            xref="container",
            y=0.85,
            yanchor="middle"
        ),
    ),
    width=800,
    height=500,
)
fig

In [383]:
fig.add_scatter(x=[data.iloc[-1][0]], y=[data.iloc[-1][1]], mode="markers", marker=dict(color="red", symbol="star", size=10), name="EERIE")
fig

In [396]:
fig.write_image("roc.svg")

In [384]:
# px.colors.named_colorscales()["rdylgn"]/
# px.colors.qualitative.swatches(["rdylgn"])px.colors.qualitative.Plotly
# px.colors.qualitative.rdylgn

In [385]:
# fig = px.scatter(x=data[0], y=data[1], color=data[2], width=800,
#                  height=500, category_orders={2: ["A", "B", "C", "D", "E", "F"]}
#                  )
# # color_continuous_scale="rdylgn",
# fig.update_layout(
#     dict1=dict(
#         legend=dict(
#             title="class"
#         ),
#         title=dict(
#             text="The Result of Classification",
#             font=dict(
#                 size=20
#             ),
#             x=0.48,  # 0 ~ 1
#             xanchor="center",
#             xref="container",
#             y=0.92,
#             yanchor="middle"
#         ),
#     ),
# )
# fig

In [386]:
# fig.add_scatter(x=[data[-1][0]], y=[data[-1][1]], mode="markers", marker=dict(color="red", symbol="star", size=15))
# fig.update_layout(
#     showlegend=False
# )
# fig

In [387]:
# data[-1][0]

In [388]:
# fig.write_image("xy.svg")

In [389]:
# df = pd.read_excel("data3.xlsx", header=0)
# df["Date"] = pd.to_datetime(df["Date"])
# df

In [390]:
# data = pd.concat((df["three_ss"], df["uniq"], df["star"]), axis=1)
# data = data.astype(float)
# data = np.array(data)
# data = pca.fit_transform(data)
# data = np.c_[data, train_clusters]
# data

In [391]:
# px.scatter(x=data[:, 0], y=data[:, 1], color=data[:, 2])

In [392]:
# def distance(p, q, f):
#     return np.sqrt(np.sum(f(p - q) ** 2))
#
#
# def efm(X, K, beta=0.5, f=np.identity):
#     m, n = X.shape
#     centers = [X[np.random.choice(m)]]
#     # for c in centers:
#     #     print(c.values)
#
#     while len(centers) < K:
#         scores = np.array([min([distance(c, x, f) for c in centers]) for x in X])
#         sum_score = np.sum(scores)
#         probs = [s / sum_score for s in scores]
#         cum_probs = np.cumsum(probs)
#
#         r = np.random.random()
#         ind = np.where(cum_probs >= r)[0][0]
#         centers.append(X[ind])
#
#     pred = np.zeros(m, dtype=int)
#
#     for i, x in enumerate(X):
#         best_center = 0
#         min_distance = distance(x, centers[0], f)
#         for j in range(1, K):
#             d = distance(x, centers[j], f)
#             if d < min_distance:
#                 min_distance = d
#                 best_center = j
#         pred[i] = best_center
#
#     for i in range(K):
#         indices = np.where(pred == i)[0]
#         if len(indices) > 0:
#             center = np.zeros(n)
#             for j in indices:
#                 center += X[j]
#             center /= len(indices)
#             centers[i] = center
#
#     while True:
#         prev_pred = pred.copy()
#         for i, x in enumerate(X):
#             best_center = 0
#             min_distance = distance(x, centers[0], f)
#             for j in range(1, K):
#                 d = distance(x, centers[j], f)
#                 if d < min_distance:
#                     min_distance = d
#                     best_center = j
#             pred[i] = best_center
#
#         if np.all(pred == prev_pred):
#             break
#
#         for i in range(K):
#             indices = np.where(pred == i)[0]
#             if len(indices) > 0:
#                 center = np.zeros(n)
#                 for j in indices:
#                     center += X[j]
#                 center /= len(indices)
#                 centers[i] = (1 - beta) * centers[i] + beta * center
#
#     return pred, centers
#
#
# for i in range(2, 10):
#     # 进行聚类
#     pred, centers = efm(data, K=i, f=lambda x: np.sqrt(np.abs(x)))
#
#     # 计算评价指标
#     sil_score = silhouette_score(data, pred)
#     ch_score = metrics.calinski_harabasz_score(data, pred)
#     db_score = davies_bouldin_score(data, pred)
#     print(i)
#     print(sil_score)
#     print(ch_score)
#     print(db_score)
#     print("---------------")

In [393]:
# from sklearn import datasets
# from sklearn.cluster import DBSCAN
# from sklearn.metrics import silhouette_score
#
# for i in range(2, 10):
#     # 进行聚类
#     # 构建DBSCAN聚类模型
#     dbscan = DBSCAN(eps=0.5, min_samples=7)
#     dbscan_labels = dbscan.fit_predict(data)
#
#     # 计算评价指标
#     sil_score = silhouette_score(data, dbscan_labels)
#     ch_score = metrics.calinski_harabasz_score(data, dbscan_labels)
#     db_score = davies_bouldin_score(data, dbscan_labels)
#     print(i)
#     print(sil_score)
#     print(ch_score)
#     print(db_score)
#     print("---------------")
#


In [394]:
# import numpy as np
#
#
# class CMeans:
#     def __init__(self, n_clusters=2, m=2, max_iter=300, tol=1e-4):
#         self.n_clusters = n_clusters
#         self.m = m
#         self.max_iter = max_iter
#         self.tol = tol
#         self.U = None
#
#     def fit(self, X):
#         n_samples, n_features = X.shape
#         # 初始化隶属度矩阵 U
#         self.U = np.random.rand(n_samples, self.n_clusters)
#         self.U = self.U / np.sum(self.U, axis=1, keepdims=True)
#         centers = np.random.rand(self.n_clusters, n_features)
#         # 开始迭代
#         for i in range(self.max_iter):
#             # 更新聚类中心
#             centers = np.dot(self.U.T, X) / np.sum(self.U, axis=0, keepdims=True).T
#             # 更新隶属度矩阵
#             d = np.linalg.norm(X[:, None, :] - centers[None, :, :], axis=2)
#             self.U = 1 / np.sum((d[:, :, None] / d[:, None, :]) ** (2 / (self.m - 1)), axis=2)
#             # 终止条件
#             if np.linalg.norm(self.U - self.U.mean(axis=0), ord='fro') < self.tol:
#                 break
#         # 将隶属度矩阵转化为聚类标签
#         labels = np.argmax(self.U, axis=1)
#         return centers, labels
#
#
# # 进行聚类并输出结果
# for i in range(2, 10):
#     cmeans = CMeans(n_clusters=i)
#     centers, labels = cmeans.fit(data)
#     # labels = cmeans.labels_
#     # print(labels)
#
#     sil_score = silhouette_score(data, labels)
#     ch_score = metrics.calinski_harabasz_score(data, labels)
#     db_score = davies_bouldin_score(data, labels)
#     print(i)
#     print(sil_score)
#     print(ch_score)
#     print(db_score)
#     print("-------------")

In [395]:
# import numpy as np
# from sklearn.cluster import KMeans
# from sklearn.metrics.pairwise import pairwise_distances
# from sklearn.metrics import pairwise_kernels
#
#
# def LRR_clustering(X, n_clusters):
#     n_samples, n_features = X.shape
#     C = np.zeros((n_samples, n_samples))
#     for i in range(n_samples):
#         C[i, i] = 1.0
#     alpha = 0.1
#     max_iter = 100
#     tol = 1e-6
#     for i in range(max_iter):
#         Z = np.linalg.inv(np.eye(n_samples) + alpha * C).dot(X)
#         Z[np.abs(Z) < tol] = 0.0
#         E = X - Z
#         C = pairwise_kernels(E, metric='rbf')
#     labels = KMeans(n_clusters=n_clusters, max_iter=100).fit(Z).predict(X)
#     return labels
#
#
# for i in range(2, 10):
#     labels = LRR_clustering(data, i)
#     sil_score = silhouette_score(data, labels)
#     ch_score = metrics.calinski_harabasz_score(data, labels)
#     db_score = davies_bouldin_score(data, labels)
#     print(i)
#     print(sil_score)
#     print(ch_score)
#     print(db_score)
#     print("-------------")
#
